In [63]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [64]:
math_df = pd.read_excel('/kaggle/input/contedos-matemtica-unicamp.xlsx')
math_df = math_df.replace({"Sim": 1, "Não": 0})

<ipython-input-64-d5ffec3949ae>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  math_df = math_df.replace({"Sim": 1, "Não": 0})


# **Cálculo de correlação entre a presença dos conteúdos, importante para escolher a melhor estratégia e modelo para o caso**

GEOMETRIA PLANA E PROGRESSÃO ARITIMÉTICA SEMPRE ESTÃO PRESENTES, LOGO SÃO CONSTANTES, O QUE TORNA IMPOSSÍVEL DE CALCULAR AS CORRELAÇÕES RELAIONADAS.


In [65]:
math_df_corr = math_df.drop(columns=["Ano", "Geometria Plana", "Progressão Aritmética"])
math_df_corr =math_df_corr.loc[:, math_df.var() > 0]

correlação = math_df_corr.corr()
correlação

,Geometria Espacial,Lógica,Sistemas Lineares,Função Quadrática,Conjuntos e Inequações,Análise Combinatória,Probabilidade,Função Composta e Geometria Analítica,Trigonometria,Matrizes,Grandezas e Regra de Três,Circunferência
Geometria Espacial,1.000000,-0.403390,-0.204973,0.314072,0.147573,-0.050461,0.073598,0.179196,-0.008476,0.188562,-0.012423,0.019035
Lógica,-0.403390,1.000000,-0.207870,-0.034126,-0.264575,-0.164488,-0.027067,-0.125988,0.172219,0.098601,0.252413,-0.386763
Sistemas Lineares,-0.204973,-0.207870,1.000000,-0.212814,0.054997,0.034192,0.092834,-0.054997,-0.101080,-0.474342,0.138889,0.007094
Função Quadrática,0.314072,-0.034126,-0.212814,1.000000,-0.084270,0.272434,0.258630,0.084270,0.154881,0.309570,0.007094,0.326087
Conjuntos e Inequações,0.147573,-0.264575,0.054997,-0.084270,1.000000,-0.046421,0.007161,0.033333,-0.128654,-0.372678,0.054997,0.102328
Análise Combinatória,-0.050461,-0.164488,0.034192,0.272434,-0.046421,1.000000,0.280484,0.046421,-0.121310,-0.116775,-0.177799,0.272434
Probabilidade,0.073598,-0.027067,0.092834,0.258630,0.007161,0.280484,1.000000,0.214834,0.241843,0.080064,0.354459,0.258630
Função Composta e Geometria Analítica,0.179196,-0.125988,-0.054997,0.084270,0.033333,0.046421,0.214834,1.000000,0.128654,-0.089443,-0.054997,-0.102328
Trigonometria,-0.008476,0.172219,-0.101080,0.154881,-0.128654,-0.121310,0.241843,0.128654,1.000000,0.047946,0.290604,-0.145201
Matrizes,0.188562,0.098601,-0.474342,0.309570,-0.372678,-0.116775,0.080064,-0.089443,0.047946,1.000000,0.070273,-0.107676


# **APLICAÇÃO E TESTE DO PRIMEIRO MODELO: REDE NEURAL DO TIPO RECORRENTE**

PRIMEIRO PASSO: MODELAGEM DOS DADOS


In [66]:
def formatar_dados(df, janela):
    X = []
    y = []

    for i in range(len(df) - janela):
        X.append(df.iloc[i:i + janela, 1:].values)
        y.append(df.iloc[i + janela, 1:].values)
    
    return np.array(X), np.array(y)

In [67]:
x,y = formatar_dados(math_df,5)

In [68]:
input_shape = x.shape

In [69]:
print(input_shape)

(26, 5, 15)


Definição de parâmetros:
 *  Otimizador: Adaptive Moment Estimation (Adam)
 *  Função de perda: Binary Cross Entropy
 *  Métrica: Acurácia
 *  Função de Ativação: SIGMOIDE (CAMADA DE OUTPUT) e Unidade LInear Retificada-RELU (CAMADAS OCULTAS)

In [70]:
from tensorflow import keras
from tensorflow.keras import layers

In [71]:
deep_model = keras.Sequential([
    layers.LSTM(30, activation='relu', return_sequences=True, input_shape=(5, 15)),
    layers.LSTM(28, activation='relu', return_sequences=True,),
    layers.LSTM(26, activation='relu', return_sequences=True,),
    layers.LSTM(22, activation='relu', return_sequences=False,),
    layers.Dense(15,activation='sigmoid'),
])

shallow_model = keras.Sequential([
    layers.LSTM(32, input_shape=(3, 15), activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [73]:
deep_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

shallow_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [74]:
deep_model.summary()
shallow_model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_45 (LSTM)                       │ (None, 5, 30)               │           5,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_46 (LSTM)                       │ (None, 5, 28)               │           6,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_47 (LSTM)                       │ (None, 5, 26)               │           5,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_48 (LSTM)                       │ (None, 22)                  │           4,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 15)                  │             345 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,505 (87.91 KB)

 Trainable params: 22,505 (87.91 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_49 (LSTM)                       │ (None, 32)                  │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,177 (24.13 KB)

 Trainable params: 6,177 (24.13 KB)

 Non-trainable params: 0 (0.00 B)

Não dividirei o conjunto de dados entre treino e validação, pois a sequência de dados é pequena e limitada.

In [88]:
deep_model.fit(
    x=x,
    y=y,
    batch_size=12,
    epochs=200,
)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5401 - loss: 0.2569
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6026 - loss: 0.2586
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3902 - loss: 0.2399
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1186 - loss: 0.2490    
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3189 - loss: 0.2412
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4199 - loss: 0.2667
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3990 - loss: 0.2452
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3381 - loss: 0.2509
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4511 - loss: 0.2634
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4599 - loss: 0.2281
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4199 - loss: 0.2465
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2804 

Em 200 épocas, os números se tornam melhores até o final do treinamento, no entanto, a acurácia é baixa, e também pode representar um possível overfit.

In [84]:
last_5 = math_df.drop(labels=['Ano'],axis=1).tail(5)
last_5

,Geometria Plana,Geometria Espacial,Lógica,Sistemas Lineares,Função Quadrática,Conjuntos e Inequações,Análise Combinatória,Probabilidade,Progressão Aritmética,Função Composta e Geometria Analítica,Trigonometria,Matrizes,Grandezas e Regra de Três,Polinômios,Circunferência
26,1,0,1,1,0,1,1,1,1,0,1,0,1,0,0
27,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0
28,1,0,1,1,0,1,0,1,1,0,1,0,1,0,0
29,1,1,1,1,0,1,0,1,1,0,1,0,1,0,0
30,1,0,1,1,0,1,0,1,1,0,1,0,1,0,0


In [89]:
input_data = np.expand_dims(last_5.values, axis=0) 
prediction = deep_model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [90]:
prediction

array([[1.0000000e+00, 9.8935854e-01, 9.9516487e-01, 9.9991482e-01,
        2.6236820e-01, 9.9950540e-01, 4.3310903e-02, 9.9999350e-01,
        1.0000000e+00, 4.0793590e-02, 1.0000000e+00, 2.1911999e-10,
        1.0000000e+00, 1.3590098e-20, 5.7154920e-02]], dtype=float32)